In [10]:
import requests
import zipfile
import os
import fnmatch

def download_file(url):
    dirpath = './data_files/'
    fname =  dirpath + url.split('/')[-1]
    r = requests.get(url, stream=True)
    with open (fname, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
                f.flush()
        return fname
    return False

def zip_extract(fname):
    dirpath = './data_files/'
    zfile = zipfile.ZipFile(fname)
    zfile.extractall(dirpath)

if __name__ == '__main__':
    #　保存ファイルの場所
    dirpath = './data_files/'
    
    # 福沢諭吉テキストurlのリストを作る！
    !grep 福沢,諭吉 aozora_word_list_utf8.csv > yukichi.csv
    text_url_list = [line.split(',')[45] for line in open('yukichi.csv')]
    
    #サーバーからデータをダウンロード && ファイルの展開
    for url in text_url_list:
        #fname = download_file(url)
        fname = './data_files/43029_ruby_23223.zip' #deback
        if fname:
            print('{} is download'.format(fname))
            zip_extract(fname)
        !sleep 1.0

    #すべて展開する
    # txt_list =[./data_files/kyoto_gakkono_ki.txt]
    txt_list = fnmatch.filter(os.listdir(dirpath), '*.txt')
    print(len(fname) == len(txt_list))
    
    #txt_list全体にdirpathを加える
    mapped_txt_list = map(lambda txt_path:dirpath+txt_path, txt_list)

./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip 

In [6]:
import codecs

# 処理済みテキストの格納
all_sjis_text = ""

# ファイル読込み、内部表現化
for text in txt_list:
    print(text)
    f = codecs.open(text, "r", "sjis")
    print(f)
    
    sjis_text = f.read()
    f.close()

    # ファイル整形
    import re
    # ヘッダ部分の除去
    sjis_text = re.split('-{5,}',sjis_text)[2]
    # フッタ部分の除去
    sjis_text = re.split('底本：',sjis_text)[0]
    # | の除去
    sjis_text = sjis_text.replace('|', '')
    # ルビの削除
    sjis_text = re.sub('《.+?》', '', sjis_text)
    # 入力注の削除
    sjis_text = re.sub('［＃.+?］', '',sjis_text)
    # 空行の削除
    sjis_text = re.sub('\n\n', '\n', sjis_text)
    sjis_text = re.sub('\u3000', '',sjis_text)
    # carrige returnの削除
    sjis_text = re.sub('\r', '', sjis_text)
    #  line feed の削除
    sjis_text = re.sub('\n', '',sjis_text)
    

    # 整形結果確認
#     print(sjis_text[:200])
    # 見やすくするため、空行 
#     print('\n\n')
    # 後ろの100文字の表示 
#     print(sjis_text[-200:])
    
    #テキストに保存
    all_sjis_text += sjis_text
print(all_sjis_text)

kyoto_gakkono_ki.txt


FileNotFoundError: [Errno 2] No such file or directory: 'kyoto_gakkono_ki.txt'

In [160]:
from janome.tokenizer import Tokenizer

# Tokenneizerインスタンスの生成 
t = Tokenizer()

# テキストを引数として、形態素解析の結果、名詞のみの配列で抽出する関数を定義 
def extract_words(sjis_text):
    tokens = t.tokenize(sjis_text)
    return [token.surface for token in tokens if  token.part_of_speech.split(',')[0] in ['名詞']]

#  関数テスト
# ret = extract_words('明治五年｜申五月｜朔日、社友｜早矢仕氏とともに京都にいたり、名所旧跡はもとよりこれを訪うに暇あらず、博覧会の見物ももと余輩上京の趣意にあらず、まず府下の学校を一覧せんとて、知る人に案内を乞い、諸処の学校に行きしに、その待遇きわめて厚く、塾舎・講堂、残るところなく見るを得たり。')
# for word in ret:
#     print(word)

# 全体のテキストを句点で区切った配列にする。
sentence_list = all_sjis_text.split('。')

# それぞれの文章を単語リストに変換
word_list = [extract_words(sentence)for sentence in sentence_list]
    
# 単語リストの結果を一部表示
for word in word_list[0]:
    print(word)


明治
五
年
申
五月
朔日
社友
早
矢
氏
京都
名所旧跡
これ
暇
博覧
会
見物
もと
余
輩
上京
趣意
府下
学校
一覧
人
案内
乞い
諸処
学校
待遇
塾舎
講堂
ところ


In [161]:
from gensim.models import word2vec

# size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
# model.wv.most_similarの結果が1に近いものばかりで、model.dict['wv']のベクトル値が小さい値ばかりの 
# ときは、学習回数が少ないと考えられます。
# その場合、iterの値を大きくして、再度学習を行います。

# 事前準備したword_listを使ってWord2Vecの学習実施
model = word2vec.Word2Vec(word_list, size=100,min_count=5,window=5,iter=100)

In [154]:
# 結果の確認1
# 一つ一つの単語は100次元のベクトルになっています。 
# 「世間」のベクトル値を確認します。
print(model.__dict__['wv']['金'])

[-0.37224346 -0.8927241  -0.48001295 -1.7234765   2.6264646  -2.1272457
  0.2847044   0.7753684   0.13785578 -1.6422555   0.22328389 -1.4586166
 -0.3154943   0.05757952 -1.5670145   1.7751298  -1.3134022  -2.4336133
 -0.11664536  0.09272543  0.1047623  -0.34785834  2.281147   -0.70943296
 -0.35234544 -0.29986173  1.1909989   0.9662525  -0.23316458 -1.946647
  0.5881884   0.73663265  0.87169886  2.0406816   0.2370829   0.6062448
 -1.7646132   0.5496129  -0.75191754  1.0534582  -1.0870793  -1.7766922
  2.7578037   0.14994285  0.10068615  0.2889882  -0.6134185  -1.5999016
  1.9930787  -0.91996723  0.89026296 -0.46296346  0.67923206  1.4639814
 -0.95738447  0.3699064   1.1375983   0.0223671   0.07891569  0.07816242
 -1.0634359   3.7946665  -1.3917263   0.56965953 -0.93708766  0.1292948
 -0.45156002  0.54583174  0.845524    0.8359749   0.15579444  1.0860866
  2.8373322   3.1709573   1.528081   -3.9184134  -2.8767745  -1.0665125
 -1.3443097  -1.479317   -0.5277039   0.29506394 -2.4047484  -0

In [168]:
# 結果の確認2
# 関数most_similarを使って「世間」の類似単語を調べます 
ret = model.wv.most_similar(positive=['命']) 
for item in ret:
    print(item[0], item[1])

どころ 0.4725894629955292
府 0.4475650489330292
上野 0.4118836522102356
強盗 0.41098856925964355
介 0.40858179330825806
受 0.4079661965370178
つもり 0.3988129496574402
役目 0.39686426520347595
暴政 0.3637733459472656
名目 0.3621255159378052


/Users/hamaguchikazuki/.pyenv/versions/3.6.5/envs/ppgVC/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
